## 1.爬取江苏概要信息

In [ ]:
#爬取江苏概要信息
import json
import requests
import pandas as pd
urlDetail = 'http://www.landjs.com/tAfficheParcel/searchParcelInfo'
url = 'http://www.landjs.com/tAfficheParcel/bulletinInfo'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'
    }

data = {
    'index':'1',
    'mrFlag': '2',
    'size': '21764' #总数据记录
   # 'xzqDm': '3201' #行政代码
}

response = requests.post(url=url, json=data, headers=headers)
jsonData = json.loads(response.text)
DFData = pd.DataFrame(jsonData['list'])
DFData.to_csv(r'F:\Users\zhang\Desktop\output.csv', index=False, encoding='utf-8')

## 2.爬取江苏土地经纬度

In [2]:
# 爬取江苏土地经纬度
import os 
import json
import time
import random
import requests
import pandas as pd


# 生成随机延迟的函数
def random_delay(min,max):
    # 设置延迟时间的范围，可以根据实际情况调整
    min_delay = min  # 最小延迟时间（秒）
    max_delay = max  # 最大延迟时间（秒）
    # 生成随机延迟时间
    delay = random.uniform(min_delay, max_delay)
    # 执行延迟
    time.sleep(delay)

    

startTime = time.time() #记录时间

#读取结果
dir = r'F:\Users\zhang\Desktop\土地爬取\江苏土地市场网(供地结果)\江苏土地市场网(供地结果).csv'
df = pd.read_csv(dir,usecols=[0,1],encoding='utf-8')
lst = df.iloc[:,1].tolist()  #dataFrame第二列转换成列表
#目标网页
url_lna = 'http://221.226.28.84:7103/map/geometry/get?id='

#headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}

fault,lng ,lat, zdGuid, gdGuid,sum_lng_lat =[], [], [], [], [], []

for i in lst:
    try:
        # 代理服务器
        proxyHost = "http-short.xiaoxiangdaili.com"
        proxyPort = "10010"

        # 代理验证信息
        proxyUser = "1050279235694252032"
        proxyPass = "XzvTx9a7"

        proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
            "host": proxyHost,
            "port": proxyPort,
            "user": proxyUser,
            "pass": proxyPass,
        }

        # 设置 http和https访问都是用HTTP代理
        proxies = {
            "http": proxyMeta,
            "https": proxyMeta,
        }

        #  设置IP切换头
        tunnel = random.randint(1, 10000)
        headers = {"Proxy-Tunnel": str(tunnel)}

        random_delay(0,1) #随机延迟    
        index = lst.index(i)
        url = url_lna + i + '&type=tg'
        response = requests.post(url=url, headers=headers, proxies=proxies)
        jsonData = json.loads(response.text)
        DFData = pd.DataFrame(jsonData['coordinates'])
        if len(DFData)>0:
            #经纬度取均值
            data_lst = DFData.transpose().iloc[:,0].tolist()  #transpose转置
            sum_lng = 0
            sum_lat = 0
            
            if type(data_lst[0][0]) == list:
                k = 0
                for x in data_lst:
                    for t in x:
                        k+=1
                        sum_lng = sum_lng + t[0]
                        sum_lat = sum_lat + t[1]
                av_lng = sum_lng / k #取均值
                av_lat = sum_lat / k
            else:
                for x in data_lst:
                    sum_lng = sum_lng + x[0]
                    sum_lat = sum_lat + x[1]
                av_lng = sum_lng / len(data_lst) #取均值
                av_lat = sum_lat / len(data_lst)   
                
            #存储进入列表
            lng.append(av_lng)
            lat.append(av_lat)
            zdGuid.append(i)
            gdGuid.append(df.iloc[index,0])
            
            #保留所有经纬度
            if len(jsonData['coordinates'])>1:
                sum_lng_lat.append([jsonData['coordinates']])
            else:
                sum_lng_lat.append(jsonData['coordinates'])
        
            print("Request {} completed，Actual records:{}!".format(i,len(lng)))     
    except Exception as e:
            print(i+"爬取失败")
            fault.append(i)


array01 = numpy.array([gdGuid, zdGuid, lng, lat, sum_lng_lat]).T
sumDF = pd.DataFrame(array01, columns=['gdGuid', 'zdGuid', 'lng','lat','sum_lng_lat'])
sumDF.to_excel(r'F:\Users\zhang\Desktop\output.xlsx', index=False)

#爬虫统计
endtime = time.time()
dtime = endtime - startTime
print("Program runtime: %.8s s" % dtime)

Request 2168722 completed，Actual records:1!
Request 1896798 completed，Actual records:2!


NameError: name 'numpy' is not defined

In [5]:
sumData.to_csv(r'F:\Users\zhang\Desktop\output.csv', index=False, encoding='utf-8')